In [2]:
import streamlit as st
import pandas as pd
import datetime

# --- LOGIN SECTION ---
def login():
    st.title("🔐 Login to Church Dashboard")

    username = st.text_input("Username")
    password = st.text_input("Password", type="password")

    if st.button("Login"):
        if username == "christmbassybremen" and password == "CEde2025":
            st.session_state["logged_in"] = True
        else:
            st.error("❌ Invalid username or password")

if "logged_in" not in st.session_state or not st.session_state["logged_in"]:
    login()
    st.stop()  # Prevent rest of the app from showing


# --- MAIN DASHBOARD ---
st.set_page_config(layout="wide")
st.title("📊 CE Germany - Church Attendance Dashboard")

st.sidebar.header("📤 Upload New Dataset")
uploaded_file = st.sidebar.file_uploader("Upload CSV", type=["csv"])

@st.cache_data
def load_data(file):
    df = pd.read_csv(file)
    df["Reporting For (Date)"] = pd.to_datetime(df["Reporting For (Date)"])
    df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors='coerce')
    return df

if uploaded_file:
    df = load_data(uploaded_file)
    last_update_note = "Last update: just now"
else:
    df = load_data("CE_baseboard/church-attendance-report2025-07-02.csv")
    last_update_note = f"Last update from file timestamp: {df['Timestamp'].max().strftime('%Y-%m-%d %H:%M') if pd.notnull(df['Timestamp'].max()) else 'N/A'}"

# --- FILTERING ---
st.sidebar.header("🔎 Filter Data")
churches = df["Church"].unique()
selected_churches = st.sidebar.multiselect("Select Church(es)", churches, default=list(churches))

min_date, max_date = df["Reporting For (Date)"].min(), df["Reporting For (Date)"].max()
date_range = st.sidebar.date_input("Select Date Range", [min_date, max_date], min_value=min_date, max_value=max_date)

# Apply filters
filtered_df = df[
    (df["Church"].isin(selected_churches)) &
    (df["Reporting For (Date)"] >= pd.to_datetime(date_range[0])) &
    (df["Reporting For (Date)"] <= pd.to_datetime(date_range[1]))
]

# --- METRICS ---
st.markdown("### 📌 Summary")
col1, col2, col3 = st.columns(3)
col1.metric("Total Attendance", int(filtered_df["Total Attendance"].sum()))
col2.metric("Total Reports", filtered_df.shape[0])
col3.metric("Latest Timestamp", df["Timestamp"].max().strftime("%Y-%m-%d %H:%M") if pd.notnull(df["Timestamp"].max()) else "N/A")

st.caption(f"📅 Data Range: {min_date.date()} to {max_date.date()}")
st.caption(last_update_note)

# --- CHARTS ---
st.subheader("📈 Attendance Over Time")
line_chart_data = filtered_df.groupby("Reporting For (Date)")["Total Attendance"].sum().reset_index()
st.line_chart(line_chart_data.rename(columns={"Reporting For (Date)": "Date"}))

st.subheader("🏛️ Attendance by Church")
church_chart = filtered_df.groupby("Church")["Total Attendance"].sum().sort_values(ascending=False)
st.bar_chart(church_chart)

st.subheader("👥 Attendance Breakdown")
breakdown_cols = ["Men- Total", "Women- Total", "Youth- Total", "Children- Total"]
breakdown_df = filtered_df[breakdown_cols].sum().reset_index()
breakdown_df.columns = ["Group", "Count"]
st.dataframe(breakdown_df.set_index("Group"))

st.info("Upload a new CSV file using the sidebar to refresh the dashboard. Columns must remain the same.")


ModuleNotFoundError: No module named 'streamlit'